# Example: CSerpentModule

CSerpentModule allows you to develop C extensions in a Jupyter notebook, modify their source code, and reload them on the fly.

## Building the C-serpent extension module

C-serpent is primarily a standalone command-line program, but it can also be built as a Python extension module so that it can be called directly from Python. In order to use cserpentmodule, you first need to build this extension.

In a UNIX-like shell, you can build the extension module using a pair of command like this:

    PY_INC_DIR="$(python -c "import distutils.sysconfig; print(distutils.sysconfig.get_python_inc())")"
    
    cc -shared -fPIC -o cserpent_py.so cserpent_py.c -I$PY_INC_DIR

The first line asks Python where it's header files are (you might need to install these, on Linux they might be in a package called python3-dev or python3-devel). The scond line builds the cserpent extension module.

## Compile and wrap C code

In [1]:
import cserpentmodule, numpy

In [2]:
c_code = """

#include <stdint.h>

void add_array_scalar_i32(int N, int32_t *x, int32_t y) {
    for(int i = 0; i < N; i++) 
        x[i] += y;
}

"""

In [3]:
m = cserpentmodule.CSerpentModule("my_c_module")
my_c_module = m.compile(c_code, ['add_array_scalar_i32'])

Running compiler:
cc -I/home/hsnyder/micromamba/envs/diffusers/include/python3.10 -I/home/hsnyder/micromamba/envs/diffusers/lib/python3.10/site-packages/numpy/core/include -Wall -Wno-unused-function -shared -fPIC -x c - -o /tmp/my_c_module_1709314149.so -O3 -fopenmp
Build successful!


In [4]:
x = numpy.zeros(5, numpy.int32)
my_c_module.add_array_scalar_i32(len(x), x, 3)
x

array([3, 3, 3, 3, 3], dtype=int32)

## Update C code and live-reload module

In [5]:
import time; time.sleep(1)
# Hot-reloads need to be spaced at least 1 second apart. 
# I'm just adding this sleep so that it works if you "clear and re-run" the notebook,
# which will generally be fast enough that the two m.compile calls occur the same second

In [6]:
updated_c_code = """

#include <stdint.h>

void add_array_scalar_i32(int N, int32_t *x, int32_t y) {
    for(int i = 0; i < N; i++) 
        x[i] += y;
}

void mul_array_scalar_i32(int N, int32_t *x, int32_t y) {
    for(int i = 0; i < N; i++) 
        x[i] *= y;
}

"""

In [7]:
my_c_module = m.compile(updated_c_code, ['add_array_scalar_i32', 'mul_array_scalar_i32'])

Running compiler:
cc -I/home/hsnyder/micromamba/envs/diffusers/include/python3.10 -I/home/hsnyder/micromamba/envs/diffusers/lib/python3.10/site-packages/numpy/core/include -I/home/hsnyder/micromamba/envs/diffusers/include/python3.10 -I/home/hsnyder/micromamba/envs/diffusers/lib/python3.10/site-packages/numpy/core/include -Wall -Wno-unused-function -shared -fPIC -x c - -o /tmp/my_c_module_1709314151.so -O3 -fopenmp
Build successful!


In [8]:
dir(my_c_module)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'add_array_scalar_i32',
 'mul_array_scalar_i32']

In [9]:
x = numpy.ones(5, numpy.int32)
my_c_module.mul_array_scalar_i32(len(x), x, 3)
x

array([3, 3, 3, 3, 3], dtype=int32)